# Google Datebase Cloud SQL for MySQL

[Google Database Cloud SQL for MySQL](https://cloud.google.com/sql/mysql).

Save, load and delete documents from `MySQL` database.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-cloud-sql-mysql-python/blob/main/docs/document_loader.ipynb)

## Pre-reqs

### Install package

In [ ]:
%pip install langchain-google-cloud-sql-mysql

In [1]:
from langchain_google_cloud_sql_mysql import MySQLEngine, MySQLLoader, MySQLDocumentSaver
from langchain_core.documents import Document
import sqlalchemy

### Setup gcloud and database

In [3]:
#@markdown Please fill in the value below with your GCP project ID and then run the cell.

# Please fill in these values.
project_id = "langchain-cloud-sql-testing" #@param {type:"string"}

# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"

# Configure gcloud.
!gcloud config set project {project_id}

#@markdown Please fill in the both the Google Cloud region and name of your Cloud SQL instance. Once filled in, run the cell.
region = "us-central1" #@param {type:"string"}
instance_id = "test-instance" #@param {type:"string"}
db_name = "test" #@param {type:"string"}

# Quick input validations.
assert region, "⚠️ Please provide a Google Cloud region"
assert instance_id, "⚠️ Please provide the name of your instance"

# check if Cloud SQL instance exists in the provided region
database_version = !gcloud sql instances describe {instance_id} --format="value(databaseVersion)"
if database_version[0].startswith("MYSQL"):
  print("Found existing MySQL Cloud SQL Instance!")
else:
  print("Creating new Cloud SQL instance...")
  password = input("Please provide a password to be used for 'postgres' database user: ")
  !gcloud sql instances create {instance_name} --database-version=MYSQL_5_7 \
    --region={region} --cpu=1 --memory=4GB --root-password={password} \
    --database-flags=cloudsql.iam_authentication=On
  !gcloud sql databases create {database_name} --instance={instance_name}


To update your Application Default Credentials quota project, use the `gcloud auth application-default set-quota-project` command.
Updated property [core/project].
Creating new Cloud SQL instance...


### Register gcloud account as IAM user of MySQL database

In [ ]:
# grant Cloud SQL Client role to authenticated user
current_user = !gcloud auth list --filter=status:ACTIVE --format="value(account)"

!gcloud projects add-iam-policy-binding {project_id} \
  --member=user:{current_user[0]} \
  --role="roles/cloudsql.client"

# Add current logged in IAM user to database
!gcloud sql users create {current_user[0]} --instance={instance_name} --type=cloud_iam_user

# Login as root to grants user permission to database
# !mysql -h 127.0.0.1

## Basic Usage

### Save documents in default table

In [2]:
engine = MySQLEngine.from_instance(
    project_id=project_id, region=region, instance=instance_id, database=db_name
)
# Create default table
table_name = "test-default" #@param {type:"string"}
with engine.connect() as conn:
    conn.execute(sqlalchemy.text(f"DROP TABLE IF EXISTS `{table_name}`"))
    conn.commit()
engine.init_document_table(table_name)


test_docs = [
    Document(
        page_content="Apple Granny Smith 150 0.99 1",
        metadata={"fruit_id": 1},
    ),
    Document(
        page_content="Banana Cavendish 200 0.59 0",
        metadata={"fruit_id": 2},
    ),
    Document(
        page_content="Orange Navel 80 1.29 1",
        metadata={"fruit_id": 3},
    ),
]
saver = MySQLDocumentSaver(engine=engine, table_name=table_name)
loader = MySQLLoader(engine=engine, table_name=table_name)

# Save documents
saver.add_documents(test_docs)

NameError: name 'project_id' is not defined

### Load documents from default table

In [21]:
loader = MySQLLoader(engine=engine, table_name=table_name)
docs = loader.load()
print("Loaded documents:", docs)


Loaded documents: [Document(page_content='Apple Granny Smith 150 0.99 1', metadata={'fruit_id': 1}), Document(page_content='Banana Cavendish 200 0.59 0', metadata={'fruit_id': 2}), Document(page_content='Orange Navel 80 1.29 1', metadata={'fruit_id': 3})]


### Load documents via query

In [31]:
loader = MySQLLoader(engine=engine, query=f"select * from `{table_name}` where JSON_EXTRACT(langchain_metadata, '$.fruit_id') = 1;")
docs = loader.load()
print("Loaded documents:", docs)

Loaded documents: [Document(page_content='Apple Granny Smith 150 0.99 1', metadata={'fruit_id': 1})]


## Advanced Usage

### Load documents with customize document page content & metadata

In [23]:
with engine.connect() as conn:
    conn.execute(sqlalchemy.text(f"DROP TABLE IF EXISTS `{table_name}`"))
    conn.commit()
    conn.execute(
        sqlalchemy.text(
            f"""
            CREATE TABLE IF NOT EXISTS `{table_name}`(
                fruit_id INT AUTO_INCREMENT PRIMARY KEY,
                fruit_name VARCHAR(100) NOT NULL,
                variety VARCHAR(50),  
                quantity_in_stock INT NOT NULL,
                price_per_unit DECIMAL(6,2) NOT NULL,
                organic TINYINT(1) NOT NULL
            )
            """
        )
    )
    conn.execute(
        sqlalchemy.text(
            f"""
            INSERT INTO `{table_name}` (fruit_name, variety, quantity_in_stock, price_per_unit, organic)
            VALUES
                ('Apple', 'Granny Smith', 150, 0.99, 1),
                ('Banana', 'Cavendish', 200, 0.59, 0),
                ('Orange', 'Navel', 80, 1.29, 1);
            """
        )
    )
    conn.commit()

loader = MySQLLoader(
    engine=engine,
    table_name=table_name,
    content_columns=[
        "variety",
        "quantity_in_stock",
        "price_per_unit",
        "organic",
    ],
    metadata_columns=["fruit_id", "fruit_name"],
)
print(loader.load())

[Document(page_content='Granny Smith 150 0.99 1', metadata={'fruit_id': 1, 'fruit_name': 'Apple'}), Document(page_content='Cavendish 200 0.59 0', metadata={'fruit_id': 2, 'fruit_name': 'Banana'}), Document(page_content='Navel 80 1.29 1', metadata={'fruit_id': 3, 'fruit_name': 'Orange'})]


### Save document into table with customized metadata

In [33]:
with engine.connect() as conn:
    conn.execute(sqlalchemy.text(f"DROP TABLE IF EXISTS `{table_name}`"))
    conn.commit()
engine.init_document_table(
    table_name, 
    metadata_columns=[
        sqlalchemy.Column(
            "fruit_name",
            sqlalchemy.UnicodeText,
            primary_key=False,
            nullable=True,
        ),
        sqlalchemy.Column(
            "organic",
            sqlalchemy.Boolean,
            primary_key=False,
            nullable=True,
        ),
    ],
    store_metadata=True
)
test_docs = [
        Document(
            page_content="Granny Smith 150 0.99",
            metadata={"fruit_id": 1, "fruit_name": "Apple", "organic": 1},
        ),
    ]
saver = MySQLDocumentSaver(engine=engine, table_name=table_name)

saver.add_documents(test_docs)
with engine.connect() as conn:
    result = conn.execute(sqlalchemy.text(f"select * from `{table_name}`;")).fetchall()
    print(result)

SyntaxError: unmatched ')' (1376673277.py, line 33)

## Customize Connection & Authentication

In [ ]:
from google.cloud.sql.connector import Connector

instance_connection_name = "langchain-cloud-sql-testing:us-central1:test-instance"  #@param {type:"string"}
iam_database_user = "loeng@google.com"  #@param {type:"string"}

def getconn() -> pymysql.Connection:
    conn = Connector().connect(  # type: ignore
        instance_connection_name,
        "pymysql",
        user=iam_database_user,
        db=db_name,
        enable_iam_auth=True,
    )
    return conn

mysql_engine = sqlalchemy.create_engine(
    "mysql+pymysql://",
    creator=getconn,
)

engine = MySQLEngine(engine=mysql_engine)